In [1]:
import math,simplekml,os
import pandas as pd
import numpy as np
import mytools

# 导入数据

In [17]:
grid = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/4-相关图层输出/5栅格图层/空洞匹配到的严重弱覆盖25808个栅格.data')

In [18]:
grid.head(1)

,a.day,a.region_name,a.grid_id,lon,lat,a.rsrp_sample_count,a.rsrp_weak,a.un_maxrsrp_sample_count,a.un_maxrsrp_weak_l2,a.te_maxrsrp_sample_count,a.te_maxrsrp_weak_l2,fgl,栅格类型
101,20200615,十堰,49SDS-803-130-100,110.790462,32.654904,227525,76242,3730,226,0,0,0.664907,严重弱覆盖栅格


In [19]:
grid.loc[grid['a.rsrp_sample_count']>=100,'有效栅格']='是'
grid.loc[grid['fgl']<0.7,'严重弱覆盖栅格']='是'
grid.loc[(grid['fgl']<0.936)&(grid['fgl']>=0.7),'弱覆盖栅格']='是'

In [20]:
tu = mytools.gisn.maps('地市区县')

In [21]:
grid_use = mytools.gisn.points_coverage_merge(grid,'lon','lat',coverage=tu,df_merge_coverage_columns=['地市','区县'])

In [22]:
grid_use.columns

Index(['a.day', 'a.region_name', 'a.grid_id', 'lon', 'lat',
       'a.rsrp_sample_count', 'a.rsrp_weak', 'a.un_maxrsrp_sample_count',
       'a.un_maxrsrp_weak_l2', 'a.te_maxrsrp_sample_count',
       'a.te_maxrsrp_weak_l2', 'fgl', '栅格类型', '有效栅格', '严重弱覆盖栅格', '弱覆盖栅格', '地市',
       '区县'],
      dtype='object')

# 参数设置

In [23]:
lon,lat,rsrp_count,fgl,grid_id,city,qx,grid_yx,grid_r,grid_yr,rsrp_weak = [
    'lon','lat','a.rsrp_sample_count','fgl','a.grid_id','地市','区县','有效栅格','弱覆盖栅格','严重弱覆盖栅格','a.rsrp_weak']

# 整理数据

In [24]:
def df_to_kml_zb(df,lon='lon',lat='lat',distance=70,id_use='a.grid_id',fgl='fgl',rsrp_count='a.rsrp_sample_count',
                 city = '城市',qx='区县',
                 de_col = ['a.grid_id','a.rsrp_sample_count','a.rsrp_weak','有效栅格','fgl','弱覆盖栅格','严重弱覆盖栅格']):
    df['colour'] = pd.cut(df[fgl],bins=[-0.1, 0.7, 0.936, 1.1],labels =['red', 'yellow', 'green'] )
    df['colour'] = df['colour'].astype('object')
    df.loc[df[rsrp_count]<100,'colour']='white'
    df['description']=''
    for inde_1, name_1 in enumerate(de_col):
        df['linshi']=de_col[inde_1]+' : '+df[de_col[inde_1]].astype('str')+'\n'
        df['description'] = df['description']+df['linshi']
    df = df.reindex(columns=[city,qx,id_use, 'list_data','colour','description',lon,lat])    
    print('颜色和标注完成')
    df['distance']=distance
    df['lon1'] = df[lon] + df['distance']*np.sin(45* np.pi/180)*180/( np.pi * 6371229 * np.cos(df[lat] * np.pi/180))
    df['lat1'] = df[lat] + df['distance']*np.cos(45* np.pi/180) / ( np.pi * 6371229 / 180)
    df['lon2'] = df[lon] + df['distance']*np.sin(225* np.pi/180)*180/( np.pi * 6371229 * np.cos(df[lat] * np.pi/180))
    df['lat2'] = df[lat] + df['distance']*np.cos(225* np.pi/180) / ( np.pi * 6371229 / 180)
    df['list_data']=[([(lon_2, lat_2), (lon_2, lat_1), (lon_1, lat_1),(lon_1, lat_2),(lon_2, lat_2)],[lon_1, lat_1,lon_2, lat_2]) 
     for lon_1, lat_1,lon_2, lat_2 in zip(df['lon1'],df['lat1'],df['lon2'],df['lat2'])]
    print('生成栅格方块完成')
    df = df.reindex(columns=[city,qx,id_use, 'list_data','colour','description'])    
    return df

In [25]:
grid2 = df_to_kml_zb(grid_use,lon=lon,lat=lat,distance=70,id_use=grid_id,fgl=fgl,rsrp_count=rsrp_count,city = city,
                qx=qx,de_col = [grid_id,rsrp_count,rsrp_weak,grid_yx,fgl,grid_r,grid_yr])

颜色和标注完成
生成栅格方块完成


# 切割数据导出数据

In [26]:
#单个文件
grid2.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/4-相关图层输出/5栅格图层/严重弱覆盖/严重弱覆盖栅格.data')
# #多个文件
# for name_g,data_g in grid2.groupby([city]):
#     data_g.to_pickle('G:/1-规划/评估/覆盖空洞/20200708第三版本/3-输出结果/图层/匹配栅格图层/空洞匹配到的栅格/空洞匹配到的栅格{}.data'.format(name_g))

In [12]:
del grid2

# 制作图层

In [27]:
def make_kml(data_t,name='红色栅格',cc='ff0000ff',xiankuan=0,namea='a.grid_id',list_data = 'list_data',description='description'):
    style = simplekml.Style()
    #style.linestyle.color = simplekml.Color.changealphaint(150, cc)  # 最终线条上色
    style.polystyle.outline = xiankuan
    style.polystyle.color = simplekml.Color.changealphaint(125, cc )  # 最终形状上色
    lod1 = simplekml.Lod(minlodpixels=3, maxlodpixels=-1,minfadeextent=None, maxfadeextent=None)
    grid_red = kml.newfolder(name=name)
    for grid,list_data ,description_str in zip(data_t[namea],data_t[list_data],data_t[description]):
        pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list_data[0])
        pol_r.description = description_str
        pol_r.altitudemode = simplekml.AltitudeMode.clamptoground
        lon_dd,lat_dd,lon1_dd,lat1_dd = list_data[1]
        latlonaltbox = simplekml.LatLonAltBox(east =lon_dd ,north=lat_dd ,south=lat1_dd ,west=lon1_dd,
                                              minaltitude=None, maxaltitude=None, altitudemode=None)
        pol_r.region.latlonaltbox = latlonaltbox
        pol_r.region.lod = lod1
        pol_r.style=style

In [28]:
f = mytools.othern.file_name_paths('G:/1-规划/评估/覆盖空洞/20200712第四版本/4-相关图层输出/5栅格图层/严重弱覆盖/')

In [29]:
for name_f in f:
    name_region = name_f.split('.')[0].split('/')[-1]
    grid_t = pd.read_pickle(name_f)
    grid_groupby = grid_t.groupby([city])
    for name_qx,data_region in grid_groupby:
        kml = simplekml.Kml()
#         if not os.path.exists('G:/1-规划/评估/软采评估/2020年1季度/输出个地市的小区和栅格图层/栅格图层/{}'.format(name_region)):
#             os.makedirs('G:/1-规划/评估/软采评估/2020年1季度/输出个地市的小区和栅格图层/栅格图层/{}'.format(name_region))
#         file = 'G:/1-规划/评估/软采评估/2020年1季度/输出个地市的小区和栅格图层/栅格图层/{}/栅格图层_{}_{}.kmz'.format(name_region,name_region,name_qx)
        file='G:/1-规划/评估/覆盖空洞/20200712第四版本/4-相关图层输出/5栅格图层/严重弱覆盖/{}.kmz'.format(name_qx)
        for name_c,data_t in data_region.groupby('colour'):
            if 'red' ==name_c:
                make_kml(data_t,name='红色栅格{}个'.format(data_t.shape[0]),cc='ff0000ff')
            elif 'green' == name_c:
                make_kml(data_t,name='绿色栅格{}个'.format(data_t.shape[0]),cc='ff008000')
            elif 'yellow' == name_c:
                make_kml(data_t,name='黄色栅格{}个'.format(data_t.shape[0]),cc='ff00ffff')
            elif 'white' == name_c:
                make_kml(data_t,name='白色栅格{}个'.format(data_t.shape[0]),cc='ffffffff')
        kml.save(file) if 'kml' in file else kml.savekmz(file, False)
        print('生成图层成功-{}'.format(name_qx))

生成图层成功-十堰
生成图层成功-咸宁
生成图层成功-天门
生成图层成功-孝感
生成图层成功-宜昌
生成图层成功-恩施
生成图层成功-武汉
生成图层成功-江汉
生成图层成功-潜江
生成图层成功-荆州
生成图层成功-荆门
生成图层成功-襄阳
生成图层成功-鄂州
生成图层成功-随州
生成图层成功-黄冈
生成图层成功-黄石
